In [1]:
%load_ext edx_jupyter

In [2]:
%%vertica
SELECT 'Connected to Vertica!'

password: ········


,?column?
0,Connected to Vertica!


## Identify the courses to analyze

In [61]:
%%vertica
DROP TABLE IF EXISTS ex_courses;

CREATE local TEMPORARY TABLE ex_courses
ON COMMIT PRESERVE ROWS
AS SELECT
    *
FROM
    business_intelligence.course_master
WHERE
    (content_language is NULL OR content_language = 'en-us')

,bookings,bookings_per_enrollment,conversion_rate,cnt_enrollments


In [62]:
%%vertica
SELECT COUNT(*) FROM ex_courses

,COUNT
0,3874


Number of courses: 3874

## Compute the content availability date for each enrollment

In [65]:
%%vertica
DROP TABLE IF EXISTS user_content_availability;

CREATE local TEMPORARY TABLE user_content_availability
ON COMMIT PRESERVE ROWS AS
SELECT
    CASE
        WHEN ec.course_start_date >= uc.first_enrollment_time THEN ec.course_start_date
        WHEN ec.course_start_date < uc.first_enrollment_time THEN DATE(uc.first_enrollment_time)
    END AS content_availability_date,
    uc.user_id,
    uc.course_id,
    uc.first_verified_enrollment_time
FROM
    production.d_user_course uc
JOIN
    ex_courses ec
ON
    ec.course_id = uc.course_id

,count


In [66]:
%%vertica
select count(*) from user_content_availability

,count
0,39445078


Number of enrollments in content_availability table: 39,445,078

# Conversion rate for Course Outline visitors

In [67]:
%%vertica
DROP TABLE IF EXISTS course_outline_events;
CREATE local TEMPORARY TABLE course_outline_events
ON COMMIT PRESERVE ROWS AS
SELECT
    user_id,
    LEFT(RIGHT(url, LENGTH(url) - 32), LENGTH(RIGHT(url, LENGTH(url) - 32)) - 8) AS computed_course_id,
    date,
    event_source,
    event_type
FROM
    experimental_events_run14.event_records
WHERE
    event_source = 'client' AND
    event_type = 'page' AND
    url LIKE 'https://courses.edx.org/courses/%/course/' AND
    --- Filter out the 190,809 rows in the table with non-numeric user_ids (out of 987,541,397 rows)
    user_id is not NULL AND
    REGEXP_LIKE(user_id, '^[0-9]+')

,count


In [68]:
%%vertica
SELECT count(*) FROM course_outline_events

,count
0,5380211


Number of course-outline page events: 5,380,211

In [69]:
%%vertica
DROP TABLE IF EXISTS ex_enrollments;

CREATE local TEMPORARY TABLE ex_enrollments
ON COMMIT PRESERVE ROWS AS
SELECT
    e.user_id,
    ca.course_id,
    ca.content_availability_date,
    ca.first_verified_enrollment_time
FROM
    course_outline_events e
JOIN
    user_content_availability ca
ON
    ca.course_id = computed_course_id
    AND e.user_id = ca.user_id
GROUP BY 1, 2, 3, 4

,count


In [70]:
%%vertica
SELECT COUNT(*) FROM ex_enrollments

,COUNT
0,950675


Number of rows in ex_enrollments (enrollments that accessed the course outline page): 950,675

In [71]:
%%vertica -> results

SELECT
    SUM(value) AS 'bookings',
    AVG(value) AS 'bookings_per_enrollment',
    SUM(did_upgrade)/COUNT(*) AS 'conversion_rate',
    COUNT(*) AS 'cnt_enrollments'
FROM
(
    SELECT
        CASE
            WHEN enr.first_verified_enrollment_time is not NULL THEN crs.course_seat_price
            ELSE 0
        END AS value,
        CASE
            WHEN enr.first_verified_enrollment_time is not NULL THEN 1
            ELSE 0
        END AS did_upgrade
    FROM
        ex_enrollments AS enr
    JOIN
        ex_courses AS crs
        ON crs.course_id = enr.course_id
) AS enr

In [72]:
results

,bookings,bookings_per_enrollment,conversion_rate,cnt_enrollments
0,5341513.0,5.618588,0.057082990598367915,950686


In [73]:
conversion_rate = float(results.loc[0].conversion_rate)

In [74]:
print('Conversion rate of Course outline enrollees: {0}%'.format(round(conversion_rate*100, 2)))

Conversion rate of Course outline enrollees: 5.71%


Conversion Rate of Enrollments that visit the Course Outline page is: 
## 5.71%